In [135]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import os
from __future__ import division
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.model_selection import ParameterGrid
from scipy.stats import entropy
import datetime
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)

In [142]:
df=pd.read_csv('data/atec_anti_fraud_train.csv', parse_dates=['date'])

In [143]:
df_all=df.loc[df['label']!=-1]
df_positive=df_all.loc[df_all['label']==1]
x_all=df_all.drop(['id','label','date'],axis=1)
x_positive=df_positive.drop(['id','label','date'],axis=1)
del df,df_all,df_positive

In [144]:
num_samples=x_all.shape[0]
num_features=x_all.shape[1]
num_positive=x_positive.shape[0]
threshhold4category=1000
ratio_positive=num_positive/num_samples

In [157]:
feature_counts={}
for i in range(num_features):
    detail={}
    detail['num_values']=x_all['f'+str(i+1)].unique().shape[0]
    if detail['num_values']<=threshhold4category:
        detail['feature_type']='categorical'
    else:
        detail['feature_type']='numerical'
        detail['std4all']=x_all['f'+str(i+1)].std()
        detail['mean4all']=x_all['f'+str(i+1)].mean()
        detail['std4positive']=x_positive['f'+str(i+1)].std()
        detail['mean4positive']=x_positive['f'+str(i+1)].mean()
    detail['count4all']=x_all['f'+str(i+1)].value_counts()
    dist=pd.Series(x_positive['f'+str(i+1)].value_counts(),index=detail['count4all'].axes)
    detail['count4positive']=dist.fillna(0)
    feature_counts['f'+str(i+1)]=detail

In [163]:
KLmetric=pd.Series(index=np.arange(num_features)+1)
for i in range(num_features):
    dist_all=feature_counts['f'+str(i+1)]['count4all']/num_samples
    dist_positive=feature_counts['f'+str(i+1)]['count4positive']/num_positive
    KLmetric[i+1]=entropy(dist_positive,dist_all)/np.log(feature_counts['f'+str(i+1)]['num_values'])

In [170]:
KLmetric.sort_values(ascending=False)

47     0.867411
46     0.692988
29     0.635688
28     0.631100
30     0.611112
         ...   
180    0.001258
174    0.000653
175    0.000595
10     0.000343
60     0.000005
Length: 297, dtype: float64

In [160]:
large_deviation={}
for i in range(num_features):
    detail=feature_counts['f'+str(i+1)]
    if detail['feature_type']=='numerical':
        large_deviation['f'+str(i+1)]=(detail['mean4all']-detail['mean4positive'])/detail['std4all']

In [161]:
large_deviation

{'f75': 0.24801408345440443,
 'f82': -0.4539001716867476,
 'f83': -0.10180952827523183,
 'f84': -0.026660404231106643,
 'f85': -0.32528282035802886,
 'f86': -0.23408108151726897}

In [171]:
detail=feature_counts['f47']

In [177]:
detail['count4all']/num_samples

0.0     0.034837
1.0     0.034604
2.0     0.000094
6.0     0.000005
4.0     0.000005
          ...   
8.0     0.000002
9.0     0.000002
5.0     0.000001
11.0    0.000001
10.0    0.000001
Name: f47, Length: 12, dtype: float64

In [178]:
detail['count4positive']/num_positive

0.0     0.002557
1.0     0.003382
2.0     0.003465
6.0     0.000412
4.0     0.000165
          ...   
8.0     0.000165
9.0     0.000165
5.0     0.000082
11.0    0.000082
10.0    0.000082
Name: f47, Length: 12, dtype: float64